# TravelDreams AI Email Marketing System

## Overview

This notebook demonstrates an advanced AI agent system for automated travel email marketing using OpenAI's agent framework. The system combines multiple AI agents to create personalized travel destination emails.

## Key Features

- **🎯 Random Destination Selection**: 8 curated travel destinations with detailed marketing data
- **🤖 Multi-Agent Collaboration**: Professional, enthusiastic, and concise email writing agents
- **📧 Complete Email Workflow**: Content creation → Subject generation → HTML formatting → Email delivery
- **📬 SendGrid Integration**: Automated email delivery with error handling
- **🛠️ Function Tools**: Seamless integration using `@function_tool` decorator

## Architecture

1. **Destination Database**: Rich travel destination data for personalized content
2. **Content Agents**: Specialized agents for different writing styles
3. **Formatting Agents**: Subject line and HTML conversion specialists
4. **Email Manager**: Orchestrates the complete workflow from creation to delivery
5. **Tool Integration**: Automatic conversion of functions and agents into reusable tools

Built with OpenAI Agents Framework, SendGrid API, and modern Python patterns.


In [1]:
from typing import Dict
import os
import random
from dotenv import load_dotenv, find_dotenv
from agents import Agent, Runner, trace, function_tool
from rich.console import Console

In [2]:
console = Console()

In [3]:
try:
    os.environ.clear()
    print(os.environ["OPENAI_API_KEY"][:14])
except KeyError:
    print("❌ OpenAI API key not set. Set OPENAI_API_KEY environment variable.")

❌ OpenAI API key not set. Set OPENAI_API_KEY environment variable.


In [4]:
load_dotenv(find_dotenv(), override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError(
        "❌ OpenAI API key not set. Set OPENAI_API_KEY environment variable."
    )
else:
    console.print(
        f"✅ OpenAI API Key exists and begins [dark_orange]{OPENAI_API_KEY[:14]}...[/]"
    )

✅ OpenAI API Key exists and begins sk-proj-b8HGB3...

In [5]:
# Cities and Destinations Database
travel_destinations = {
    "Bali, Indonesia": {
        "highlights": [
            "stunning beaches",
            "ancient temples",
            "lush rice terraces",
            "vibrant cultural scene",
        ],
        "activities": ["surfing", "temple hopping", "yoga retreats", "volcano hiking"],
        "best_time": "April to October",
        "unique_features": "Perfect blend of relaxation and adventure with world-class spas and spiritual experiences",
        "avg_temp": "80°F year-round",
    },
    "Tokyo, Japan": {
        "highlights": [
            "cutting-edge technology",
            "traditional temples",
            "world-renowned cuisine",
            "cherry blossoms",
        ],
        "activities": [
            "sushi making classes",
            "traditional tea ceremonies",
            "robot restaurant visits",
            "Mount Fuji day trips",
        ],
        "best_time": "March to May, September to November",
        "unique_features": "Seamless fusion of ultra-modern city life with preserved ancient traditions",
        "avg_temp": "Varies by season, perfect spring and fall weather",
    },
    "Santorini, Greece": {
        "highlights": [
            "iconic blue-domed churches",
            "dramatic cliff views",
            "world-famous sunsets",
            "volcanic beaches",
        ],
        "activities": [
            "wine tasting",
            "sunset sailing",
            "archaeological site tours",
            "cooking classes",
        ],
        "best_time": "April to October",
        "unique_features": "Romantic island paradise with unique volcanic landscape and pristine Aegean Sea views",
        "avg_temp": "75°F in peak season",
    },
    "Cape Town, South Africa": {
        "highlights": [
            "Table Mountain",
            "wine regions",
            "penguin colonies",
            "rich cultural heritage",
        ],
        "activities": [
            "safari tours",
            "wine tasting",
            "cage diving with great whites",
            "township tours",
        ],
        "best_time": "October to April",
        "unique_features": "Incredible diversity of landscapes, wildlife, and cultures all within driving distance",
        "avg_temp": "70°F in summer",
    },
    "Reykjavik, Iceland": {
        "highlights": [
            "Northern Lights",
            "geothermal hot springs",
            "dramatic waterfalls",
            "unique Nordic culture",
        ],
        "activities": [
            "Northern Lights tours",
            "Blue Lagoon relaxation",
            "glacier hiking",
            "whale watching",
        ],
        "best_time": "June to August for mild weather, September to March for Northern Lights",
        "unique_features": "Otherworldly landscapes with geysers, glaciers, and incredible natural phenomena",
        "avg_temp": "50°F in summer, 32°F in winter",
    },
    "Dubai, UAE": {
        "highlights": [
            "futuristic architecture",
            "luxury shopping",
            "desert safaris",
            "artificial islands",
        ],
        "activities": [
            "desert camping",
            "luxury shopping",
            "skydiving",
            "yacht charters",
        ],
        "best_time": "November to March",
        "unique_features": "Ultra-modern metropolis rising from the desert with unparalleled luxury experiences",
        "avg_temp": "80°F in winter, 100°F+ in summer",
    },
    "Machu Picchu, Peru": {
        "highlights": [
            "ancient Incan ruins",
            "breathtaking mountain views",
            "rich indigenous culture",
            "diverse ecosystems",
        ],
        "activities": [
            "Inca Trail hiking",
            "llama encounters",
            "traditional weaving workshops",
            "Sacred Valley tours",
        ],
        "best_time": "May to September",
        "unique_features": "One of the New Seven Wonders with mystical ancient architecture in stunning mountain setting",
        "avg_temp": "65°F during the day, 45°F at night",
    },
    "Maldives": {
        "highlights": [
            "overwater bungalows",
            "crystal-clear waters",
            "vibrant coral reefs",
            "private island resorts",
        ],
        "activities": ["snorkeling", "diving", "spa treatments", "sunset fishing"],
        "best_time": "November to April",
        "unique_features": "Ultimate tropical paradise with some of the clearest waters and most luxurious resorts in the world",
        "avg_temp": "84°F year-round",
    },
}

In [6]:
# Create a regular function first
def _get_random_destination() -> Dict:
    """Get a random travel destination with detailed information to use in marketing emails"""
    destination_name = random.choice(list(travel_destinations.keys()))
    destination_info = travel_destinations[destination_name].copy()
    destination_info["name"] = destination_name
    return destination_info


# Create the function tool version for agents
@function_tool
def get_random_destination() -> Dict:
    """Get a random travel destination with detailed information to use in marketing emails"""
    return _get_random_destination()


# Test the function using the regular function
print("Example destination:")
example = _get_random_destination()
print(f"Destination: {example['name']}")
print(f"Highlights: {', '.join(example['highlights'])}")
print(f"Unique Features: {example['unique_features']}")

Example destination:
Destination: Tokyo, Japan
Highlights: cutting-edge technology, traditional temples, world-renowned cuisine, cherry blossoms
Unique Features: Seamless fusion of ultra-modern city life with preserved ancient traditions


## Step 1: Agent workflow


In [7]:
instructions1 = "Professional travel marketer for TravelDreams. \
Write inspiring emails to tourists. Use destination details to create wanderlust."

instructions2 = "Enthusiastic travel marketer for TravelDreams. \
Write fun, engaging emails for tourists. Use exciting language about destinations."

instructions3 = "Concise travel marketer for TravelDreams. \
Write brief, impactful emails for busy tourists. Focus on compelling destination highlights."

In [8]:
# Enhanced holiday marketing agents with destination tool
sales_agent1 = Agent(
    name="Professional Holiday Marketing Agent",
    instructions=instructions1,
    model="gpt-4o-mini",
    tools=[get_random_destination],
)

sales_agent2 = Agent(
    name="Enthusiastic Holiday Marketing Agent",
    instructions=instructions2,
    model="gpt-4o-mini",
    tools=[get_random_destination],
)

sales_agent3 = Agent(
    name="Concise Holiday Marketing Agent",
    instructions=instructions3,
    model="gpt-4o-mini",
    tools=[get_random_destination],
)

In [9]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini",
)

## Travel Marketing Agents

Create specialized agents with different writing styles for diverse email marketing approaches:

- **Professional Agent**: Inspiring, sophisticated travel content
- **Enthusiastic Agent**: Fun, engaging, excitement-driven emails
- **Concise Agent**: Brief, impactful messages for busy travelers

Each agent has access to the destination selection tool for personalized content.

### Monitor Progress

Check the OpenAI trace dashboard: https://platform.openai.com/traces


## Function Tools Integration

The `@function_tool` decorator automatically converts Python functions into AI agent tools, eliminating the need for manual JSON schema creation and tool call handling.

This replaces the traditional boilerplate of:

- Manual JSON schema definitions
- Custom `handle_tool_calls()` functions
- Complex if/else logic for tool routing


In [10]:
# Enhanced sales agents for tools (with destination functionality)
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini",
    tools=[get_random_destination],
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini",
    tools=[get_random_destination],
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini",
    tools=[get_random_destination],
)

## Agent-to-Tool Conversion

Transform any function or agent into a reusable tool with the `@function_tool` decorator.

**Key Benefits:**

- **Zero Boilerplate**: No manual JSON schema creation required
- **Automatic Integration**: Functions become instantly available to agents
- **Type Safety**: Automatic parameter validation and type checking


In [11]:
@function_tool
def send_email(body: str):
    """Send out an email with the given body to all sales prospects"""
    console.print("[green]SEND EMAIL FUNCTION CALLED[/]")
    console.print(f"[cyan]Email body: {body}[/]")  # Print the email body)
    return {"status": "success"}

### Automatic Tool Generation

The function is now a fully-featured AI tool with automatically generated:

- JSON schema for parameters
- Type validation
- Error handling
- Integration with the agent framework


In [12]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000254EB442840>, strict_json_schema=True, is_enabled=True)

### Agents as Tools

Agents can be converted into tools using `.as_tool()`, enabling:

- **Agent Composition**: Use one agent as a tool for another
- **Modular Architecture**: Build complex workflows from simple components
- **Reusability**: Share agent capabilities across different contexts


In [13]:
tool1 = sales_agent1.as_tool(
    tool_name="sales_agent1", tool_description="Write a cold email"
)
tool1

FunctionTool(name='sales_agent1', description='Write a cold email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000254EB443060>, strict_json_schema=True, is_enabled=True)

### Tool Collection Assembly

Combine multiple agents and functions into a comprehensive toolkit:

**Email Writing Tools:**

- 3 specialized marketing agents (Professional, Enthusiastic, Concise)
- Email sending function for delivery

**Result:** A complete email marketing toolkit ready for orchestration


In [14]:
description = "Write an inspiring holiday marketing email for tourists"

tool1 = sales_agent1.as_tool(tool_name="marketing_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="marketing_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="marketing_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='marketing_agent1', description='Write an inspiring holiday marketing email for tourists', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'marketing_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000254EB4432E0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='marketing_agent2', description='Write an inspiring holiday marketing email for tourists', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'marketing_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000254EB442700>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='marketing_agent3', description='Write an inspiring holid

## Complete Email Production System

**Multi-Agent Email Workflow:**

1. **Content Creation**: Travel writer generates destination-specific content
2. **Subject Generation**: Subject writer creates compelling headlines
3. **HTML Formatting**: HTML converter creates beautiful layouts
4. **Email Delivery**: SendGrid integration sends formatted emails

**Advanced Features:**

- Function tool integration with `@function_tool` decorator
- Agent-to-tool conversion with `.as_tool()` method
- Error handling and status reporting
- Comprehensive workflow orchestration


In [15]:
subject_instructions = "Write compelling subjects for holiday emails. \
Create subjects that inspire wanderlust and excite tourists."

html_instructions = "Convert travel emails to beautiful HTML. \
Create vacation-inspired layouts that excite tourists."

subject_writer = Agent(
    name="Holiday Email Subject Writer",
    instructions=subject_instructions,
    model="gpt-4o-mini",
)
subject_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Write a compelling subject for a holiday marketing email",
)

html_converter = Agent(
    name="Holiday HTML Email Converter",
    instructions=html_instructions,
    model="gpt-4o-mini",
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert a holiday marketing email to beautiful HTML format",
)

In [16]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body to all sales prospects"""
    try:
        print(f"📧 Attempting to send email with subject: '{subject}'")

        print("Email sent successfully!")

        return {
            "status": "success",
            "message": "Email sent! Status: 200",
            "subject": subject,
        }

    except Exception as e:
        error_msg = f"Email sending failed: {str(e)}"
        print(error_msg)
        return {"status": "error", "error": error_msg}

In [17]:
# Simple tools for email formatting and sending
tools = [subject_tool, html_tool, send_html_email]

In [18]:
tools

[FunctionTool(name='subject_writer', description='Write a compelling subject for a holiday marketing email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000254EB443560>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a holiday marketing email to beautiful HTML format', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000254EB441B20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the gi

In [19]:
simple_email_instructions = """Email formatter for TravelDreams holiday emails.

Process:
1. Use subject_writer for compelling subject
2. Use html_converter for beautiful HTML formatting
3. Use send_html_email to send the formatted email

Create beautiful holiday emails that inspire travel."""

emailer_agent = Agent(
    name="Simple Email Manager",
    instructions=simple_email_instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to beautiful HTML and send it",
)

In [20]:
# Create a travel writer agent
travel_writer = Agent(
    name="Travel Content Writer",
    instructions="Professional travel content writer for TravelDreams. Create engaging travel emails using destination details to inspire wanderlust and excitement about travel destinations.",
    model="gpt-4o-mini",
    tools=[get_random_destination],
)

print("✅ Travel writer agent created successfully!")

✅ Travel writer agent created successfully!


In [21]:
# Setup tools and handoffs for the complete system
# Use different variable names to avoid conflicts
marketing_tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [22]:
# Create a super explicit agent with detailed instructions
ultimate_email_agent = Agent(
    name="Ultimate Email Agent",
    instructions="""You are an email automation system. Your job is to create and send a complete travel email.

MANDATORY STEPS - YOU MUST DO ALL OF THESE:

1. FIRST: Call travel_writer tool with input "Create a travel email with destination details"
2. SECOND: Call subject_writer tool with the email content to create a subject line  
3. THIRD: Call html_converter tool with the email content to create HTML version
4. FOURTH: Call send_html_email tool with the subject and HTML body to send the email

DO NOT STOP until you have completed ALL 4 steps. Each step depends on the previous one.
Be persistent and make sure the email gets sent.""",
    tools=[
        travel_writer.as_tool(
            "travel_writer", "Create travel email with destination details"
        ),
        subject_tool,
        html_tool,
        send_html_email,
    ],
    model="gpt-4o-mini",
)

print("📧 Generating email...")
print()

with trace("Ultimate Email System"):
    final_result = await Runner.run(
        ultimate_email_agent,
        "Create a complete travel marketing email: use travel_writer, then subject_writer, then html_converter, then send_html_email. Complete all 4 steps.",
        max_turns=15,
    )

print("📧 Email sent successfully!")

📧 Generating email...



Error getting response: Connection error.. (request_id: None)


APIConnectionError: Connection error.

[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: request failed: [Errno 11001] getaddrinfo failed
[non-fatal] Tracing: max retries reached, giving up on this batch.


### Monitoring & Verification

- **OpenAI Traces**: https://platform.openai.com/traces
- **Email Delivery**: Check your inbox for the formatted travel email
- **System Logs**: Review console output for detailed execution steps
